In [35]:
import os
import polars as pl

In [36]:
data_dir = 'data' 
# data_dir = '/kaggle/input/cmi-detect-behavior-with-sensor-data'
df_train = pl.read_csv(data_dir + '/train.csv')
df_test = pl.read_csv(data_dir + '/test.csv')

demo_train = pl.read_csv(data_dir + '/train_demographics.csv')
demo_test = pl.read_csv(data_dir + '/test_demographics.csv')

In [37]:
def preprocess(df, train=False):
    if train:
        df = df[:, [4, 2, 3, 9, 10, 11, 12, 13, 14, 15, 7, 8, 1]]
    else:
        df = df[:, [3, 1, 2, 4, 5, 6, 7, 8, 9, 10]]
    df = df.with_columns(pl.col(pl.Float64).fill_null(0))
    return df
    
df1 = preprocess(df_train, train=True)
df2 = preprocess(df_test, train=False)
# labels = df1.filter(pl.col("sequence_type") == "Target")["gesture"].unique().to_list() + ['Non-Target']
labels = df1["gesture"].unique()

print('BFRB', df1.filter(pl.col("sequence_type") == "Target")["gesture"].unique())
print('Non-BFRB', df1.filter(pl.col("sequence_type") == "Non-Target")["gesture"].unique())
print('Labels:', labels)
print(f'Train data: {df_train.shape} -> {df1.shape}')
print(f'Test data: {df_test.shape} -> {df2.shape}')
print(df1.head(), df2.head())

BFRB shape: (8,)
Series: 'gesture' [str]
[
	"Neck - pinch skin"
	"Above ear - pull hair"
	"Neck - scratch"
	"Eyebrow - pull hair"
	"Eyelash - pull hair"
	"Forehead - scratch"
	"Forehead - pull hairline"
	"Cheek - pinch skin"
]
Non-BFRB shape: (10,)
Series: 'gesture' [str]
[
	"Pull air toward your face"
	"Feel around in tray and pull o…
	"Drink from bottle/cup"
	"Text on phone"
	"Write name in air"
	"Write name on leg"
	"Glasses on/off"
	"Pinch knee/leg skin"
	"Wave hello"
	"Scratch knee/leg skin"
]
Labels: shape: (18,)
Series: 'gesture' [str]
[
	"Forehead - scratch"
	"Pinch knee/leg skin"
	"Forehead - pull hairline"
	"Glasses on/off"
	"Write name on leg"
	…
	"Drink from bottle/cup"
	"Scratch knee/leg skin"
	"Pull air toward your face"
	"Eyelash - pull hair"
	"Write name in air"
]
Train data: (574945, 341) -> (574945, 13)
Test data: (107, 336) -> (107, 10)
shape: (5, 13)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ subject   ┆ se

In [38]:
df1.describe()

statistic,subject,sequence_id,sequence_counter,acc_x,acc_y,acc_z,rot_w,rot_x,rot_y,rot_z,phase,gesture,sequence_type
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str
"""count""","""574945""","""574945""",574945.0,574945.0,574945.0,574945.0,574945.0,574945.0,574945.0,574945.0,"""574945""","""574945""","""574945"""
"""null_count""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0""","""0""","""0"""
"""mean""",null,null,43.645234,1.63998,1.790704,-0.459811,0.358061,-0.119146,-0.059568,-0.187089,null,null,null
"""std""",null,null,45.717673,5.781259,5.003945,6.09649,0.226847,0.464122,0.541303,0.502741,null,null,null
"""min""","""SUBJ_000206""","""SEQ_000007""",0.0,-34.585938,-24.402344,-42.855469,0.0,-0.999146,-0.999695,-0.998169,"""Gesture""","""Above ear - pull hair""","""Non-Target"""
"""25%""",null,null,17.0,-2.964844,-2.121094,-5.417969,0.177063,-0.453979,-0.509216,-0.625366,null,null,null
"""50%""",null,null,35.0,2.972656,0.6953125,-1.5625,0.338318,-0.183838,-0.107544,-0.259338,null,null,null
"""75%""",null,null,54.0,6.34375,6.816406,5.1640625,0.50238,0.200745,0.435425,0.246826,null,null,null
"""max""","""SUBJ_064387""","""SEQ_065531""",699.0,46.328125,27.183594,30.078125,0.99939,0.999817,0.999451,0.999878,"""Transition""","""Write name on leg""","""Target"""


In [39]:
demo_test, demo_train

(shape: (2, 8)
 ┌─────────────┬─────────────┬─────┬─────┬────────────┬───────────┬────────────────┬────────────────┐
 │ subject     ┆ adult_child ┆ age ┆ sex ┆ handedness ┆ height_cm ┆ shoulder_to_wr ┆ elbow_to_wrist │
 │ ---         ┆ ---         ┆ --- ┆ --- ┆ ---        ┆ ---       ┆ ist_cm         ┆ _cm            │
 │ str         ┆ i64         ┆ i64 ┆ i64 ┆ i64        ┆ f64       ┆ ---            ┆ ---            │
 │             ┆             ┆     ┆     ┆            ┆           ┆ i64            ┆ f64            │
 ╞═════════════╪═════════════╪═════╪═════╪════════════╪═══════════╪════════════════╪════════════════╡
 │ SUBJ_016452 ┆ 1           ┆ 25  ┆ 1   ┆ 1          ┆ 165.0     ┆ 52             ┆ 23.0           │
 │ SUBJ_055840 ┆ 0           ┆ 13  ┆ 0   ┆ 1          ┆ 177.0     ┆ 52             ┆ 27.0           │
 └─────────────┴─────────────┴─────┴─────┴────────────┴───────────┴────────────────┴────────────────┘,
 shape: (81, 8)
 ┌─────────────┬─────────────┬─────┬─────┬────────

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# labels = df1.filter(pl.col("sequence_type") == "Target")["gesture"].unique().to_list() + ['Non-Target']
labels = df1["gesture"].unique().to_list()

def oneHotEncode(labels, label):
    return torch.tensor([1 if label == i else 0 for i in labels], dtype=torch.float32)

def oneHotDecode(labels, encoding):
    return labels[encoding.argmax()]

def standardScalar(data):
    mean = data.mean(0, keepdim=True)
    std = data.std(0, unbiased=False, keepdim=True)
    return (data - mean) / (std + 1e-6)
    
def dataloader(df):
    ''' Works only for training data (data with ground truth) '''
    for _, grouped in df.group_by(["sequence_id", "subject"]):
        input = grouped[:, 3:10].to_torch(dtype=pl.Float32)
        target = None
        if grouped.shape[1] == 13:
            target = grouped[0, [-2, -1]]
            target = target[0, 0] # if target[0, 1] == 'Target' else target[0, 1]
            target = oneHotEncode(labels, target)
        yield { "input": input, "target": target }

data1, data2 = next(dataloader(df1)), next(dataloader(df2))
data1["input"].shape, data1["target"], data2["input"].shape, data2["target"]

(torch.Size([74, 7]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 torch.Size([56, 7]),
 None)

In [41]:
import pickle

with open('outputs/train_data.pkl', 'wb') as f:
    pickle.dump(list(dataloader(df1)), f)

with open('outputs/test_data.pkl', 'wb') as f:
    pickle.dump(list(dataloader(df2)), f)
    
with open('outputs/labels.pkl', 'wb') as f:
    pickle.dump(labels, f)